In [1]:
import time

In [2]:
from juggling_dlx_milp import *

/home/josue/Documents/git/Jugglyng1/Expérimentations/Juggling DLX/DLX/dlxm.py:69: RuntimeWarning: class "DLX_M::AbstrItem" has no virtual destructor
  class ConcItem(_AbstrItem):


### Contraintes

In [3]:
nb_hands = 2
max_height = 1 * 2
max_weight = 1
# forbidden_multiplex = [(1, 2), (1, 3), (1, 4), (2, )]
# forbidden_multiplex = [(2, 4), (2, 6), (2, 8), (2, 10), (4, ), (1, ), (3, ), (5, ), (7, ), (9, )]
forbidden_multiplex = [(1, )]
multiple_throws = True

### Musique à jouer

In [4]:
music = [( 1, "do"), ( 2, "do"), ( 3, "do"), 
         ( 4, "re"), ( 5, "mi"), ( 7, "re"), 
         ( 9, "do"), (10, "mi"), (11, "re"),
         (12, "re"), (13, "do")]

In [5]:
music = [(1, "re5"), (2, "do5"), (3, "re5"), (4, "do5"),
         (5, "re5"), (7, "do5"), (9, "mi5"),
         (11, "re5"), (12, "fa5"), (13, "la5"), (14, "re6"), (15, "fa6")]

In [6]:
music = [( 1, "do"), ( 2, "do")]

In [7]:
music = [( 1, "do")]

In [8]:
music = adapt_music(max_height, music)

In [9]:
music

[(4, 'do')]

### Algorithme

In [10]:
balls, throws = music_to_throws(music)

In [11]:
ec_instance = throws_to_extended_exact_cover(balls, throws, nb_hands, max_height, max_weight,
                                             forbidden_multiplex, multiple_throws)

4


In [12]:
dlx = dlx_solver_instance(ec_instance)

In [13]:
# for r in range(len(dlx.rows)):
#     print(dlx.row_obj(r))
len(dlx.rows)

64

In [14]:
ec_instance.colors

['', 'false', 'true', 'do', (0, 0), (1, 0)]

In [15]:
sol = get_solution_with_dlx(ec_instance)

In [16]:
len(sol.rows)

0

In [59]:
juggling_sol = process_dlx_solution(sol)

IndexError: list index out of range

In [32]:
juggling_sol.hand_config

NameError: name 'juggling_sol' is not defined

In [18]:
juggling_sol.flying

[set(), {'do'}, {'do'}, {'do'}, {'do'}, set(), set()]

In [19]:
juggling_sol.throws

[[('do', 5, 0)], [], [], [], [], [('do', 1, 0)], []]

In [20]:
print_juggling(sol)

{'do'} {} : 
{'do'} {} : 
{'do'} {} : do -- 10 --> 0
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
{'do'} {} : do -- 2 --> ?
  {}   {} : 


In [21]:
print_juggling1(juggling_sol)

[do] [] : do -- 5 --> 0
 []  [] : 
 []  [] : 
 []  [] : 
 []  [] : 
[do] [] : do -- 1 --> 0
[do] [] : 


In [17]:
ec_instance.rows[0]

[x(throw: Throw(ball='do', time=0, max_height=4), hand: 0, flying_time: 2),
 l(throw: Throw(ball='do', time=0, max_height=4)),
 (p(time: 2, hand: 0, pos: 0), 3),
 (s(time: 2, hand: 0, pos: 0), 2),
 n(time: 2, hand: 0, pos: 0),
 b(time: 2, ball: do),
 (v(time: 2, ball: do), 4),
 (f(time: 2, ball: do), 1),
 (f(time: 0, ball: do), 1),
 (f(time: 1, ball: do), 1),
 b(time: 3, ball: do),
 (f(time: 3, ball: do), 2),
 (s(time: 4, hand: 0, pos: 0), 2),
 (p(time: 4, hand: 0, pos: 0), 3)]

In [19]:
ec_instance.colors

['', 'false', 'true', 'do', (0, 0), (1, 0)]

In [18]:
ec_instance.rows

[[x(throw: Throw(ball='do', time=0, max_height=4), hand: 0, flying_time: 2),
  l(throw: Throw(ball='do', time=0, max_height=4)),
  (p(time: 2, hand: 0, pos: 0), 3),
  (s(time: 2, hand: 0, pos: 0), 2),
  n(time: 2, hand: 0, pos: 0),
  b(time: 2, ball: do),
  (v(time: 2, ball: do), 4),
  (f(time: 2, ball: do), 1),
  (f(time: 0, ball: do), 1),
  (f(time: 1, ball: do), 1),
  b(time: 3, ball: do),
  (f(time: 3, ball: do), 2),
  (s(time: 4, hand: 0, pos: 0), 2),
  (p(time: 4, hand: 0, pos: 0), 3)],
 [x(throw: Throw(ball='do', time=0, max_height=4), hand: 0, flying_time: 2),
  l(throw: Throw(ball='do', time=0, max_height=4)),
  (p(time: 2, hand: 0, pos: 0), 3),
  (s(time: 2, hand: 0, pos: 0), 2),
  n(time: 2, hand: 0, pos: 0),
  b(time: 2, ball: do),
  (v(time: 2, ball: do), 4),
  (f(time: 2, ball: do), 1),
  (f(time: 0, ball: do), 1),
  (f(time: 1, ball: do), 1),
  b(time: 3, ball: do),
  (f(time: 3, ball: do), 2),
  (s(time: 4, hand: 1, pos: 0), 2),
  (p(time: 4, hand: 1, pos: 0), 3)],
 [x(

In [17]:
t = time.time()
sols = all_solutions_with_dlx(ec_instance)
print("{} solutions trouvées en {} secondes".format(len(sols), time.time() - t))

0 solutions trouvées en 0.11350679397583008 secondes


In [18]:
sols[0].rows

IndexError: list index out of range

In [27]:
for sol in sols:
    print_juggling(sol)

{'do'} {} : 
{'do'} {} : 
{'do'} {} : do -- 10 --> 0
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
{'do'} {} : do -- 2 --> ?
  {}   {} : 
{'do'} {} : 
{'do'} {} : 
{'do'} {} : do -- 10 --> 0
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
{'do'} {} : do -- 2 --> ?
  {}   {} : 
{'do'} {} : 
{'do'} {} : 
{'do'} {} : do -- 10 --> 0
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
{'do'} {} : do -- 2 --> ?
  {}   {} : 
{'do'} {} : 
{'do'} {} : 
{'do'} {} : do -- 10 --> 0
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
{'do'} {} : do -- 2 --> ?
  {}   {} : 
{'do'} {} : 
{'do'} {} : 
{'do'} {} : do -- 10 --> 0
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   {} : 
  {}   

In [18]:
import modele
import ipywidgets
import pythreejs

# balls = [{"color":"blue", "tone":"do5", "name":"do5"},
#          {"color":"red", "tone":"re5", "name":"re5"},
#          {"color":"green", "tone":"mi5", "name":"mi5"},
#          {"color":"yellow", "tone":"fa5", "name":"fa5"},
#          {"color":"yellow", "tone":"la5", "name":"la5"},
#          {"color":"yellow", "tone":"re6", "name":"re5"},
#          {"color":"yellow", "tone":"fa6", "name":"fa6"}]

balls = [{"color":"blue", "tone":"do", "name":"do"},
         {"color":"red", "tone":"re", "name":"re"},
         {"color":"green", "tone":"mi", "name":"mi"}]

pattern = juggling_sol_to_simulator(sols[0])

sides = [-1, 1]

model = modele.Model(balls, pattern)
slider = ipywidgets.FloatSlider(min=0, max=40, step=0.1)
view = modele.View(model, sides)
slider.observe(lambda change: view.update(change['new'], change['old']), names="value")
ipywidgets.VBox([view.widget,slider])